<a href="https://colab.research.google.com/github/BakiBakiSudo61/BoTorch-and-GPytorch/blob/main/BoTorch%E3%81%A8GPyTorch%E3%81%AB%E8%A7%A6%E3%82%8C%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学習プラン：究極の柔軟性！BoTorch & GPyTorchを使いこなす with gemini2.5pro
<dl> <strong>1. 役割分担を理解する：なぜ2つもライブラリが必要？</strong>

* **GPyTorch**が「サロゲートモデル（ガウス過程）」を専門に作るエンジニア。

* **BoTorch**が、そのモデルを使って「獲得関数」を計算し、次の一手を決める戦略家。

* この分業体制が、なぜパワフルなのかを理解します。

<dt><strong>2. 新しいワークフロー：手動で最適化サイクルを回す</strong></dt>

* gp_minimizeが自動でやっていた最適化のループを、今度は手動で一つずつ組み立てます。「モデルを学習→獲得関数を最大化→次の点を評価」という流れをコードでどう書くのか見ていきましょう。

<dt><strong>3. 実践：1次元関数で最初のBoTorchコードを動かす</strong></dt>

* 慣れ親しんだ1次元関数を題材に、実際にモデルを定義し、最適化ループを回すコードを実行してみます。
</dl>

___

<strong>1. 役割分担を理解する：GPyTorchとBoTorch</strong>

scikit-optimizeでは、<code>gp_minimize</code>という一つの関数が全ての作業を内部でやってくれていました。しかし、BoTorchとGPyTorchは、仕事をきっちり分担します。
<dl>
 <strong>- GPyTorch (The Engine - ガウス過程モデル)</strong>

<dd>- PyTorchをベースにしており、非常に柔軟でモダンなガウス過程モデルを構築できます。</dd>

<dd>- GPUを使って学習を高速化したり、ディープラーニングと組み合わせたりといった高度な芸当も可能です。</dd>

<dd>- 役割: データから「予測地図（サロゲートモデル）」を作ることだけに集中します。</dd>

<strong>- BoTorch (The Driver - ベイズ最適化ロジック)</strong>

<dd>- GPyTorchが作ったモデルを入力として受け取ります。</dd>

<dd>- EIやUCBといった様々な「獲得関数」を計算し、次に試すべき最も有望な点を見つけ出します。</dd>

<dd>- 役割: 予測地図を見て、「次の一手」を決める戦略を立てることに集中します。</dd>
</dl>
この分業により、「モデルの作り方」と「次の手の決め方」を独立して、非常に細かくカスタマイズできるのが最大の強みです。


___

<strong>2. 新しいワークフロー</strong>


gp_minimizeとは違い、最適化のループを自分でfor文で書く必要があります。大まかな流れは以下のようになります。

1. 初期データを準備する:
<dl>
<dd>目的関数をいくつかの点で評価し、学習データ（train_X, train_Y）を用意します。</dd>

2. for ループで最適化サイクルを回す:

<dd>(a) モデルの学習 (GPyTorch): 現在の学習データを使って、ガウス過程モデルを学習させます。

(b) 獲得関数の定義 (BoTorch): 学習させたモデルを使って、獲得関数（例: EI）を定義します。

(c) 次の候補点の探索 (BoTorch): 獲得関数が最大になる点を探し出し、それを次の候補点 candidate とします。

(d) 目的関数の評価: candidate で目的関数を評価し、新しい (x, y) のペアを得ます。

(e) データの更新: 新しいペアを学習データに追加し、次のループに進みます。</dd>
</dl>
scikit-optimizeが隠してくれていたこのサイクルを、自分でコードとして書くことで、最適化の各ステップで何が起きているかを完全に把握できるのです。


___

<strong>3. 実践：最初のBoTorchコード</strong>

では、実際にコードを見てみましょう。以前のscikit-optimizeのコードより長くて複雑に見えますが、上記のワークフローと見比べながら読むと、やっていることは同じだと分かります。

<strong>注意: BoTorchとGPyTorchは、データをPyTorchテンソルという形式で扱います。<code>np.array</code>を<code>torch.tensor</code>に変換する部分がコードに出てきます。</strong>

▼ 最初に、ライブラリをインストールします。

In [1]:
!pip install botorch gpytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.9/779.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.0 MB/s eta 0:00:00


▼ そして、最適化を実行します。

In [15]:
import torch
import numpy as np

# --- 1. 目的関数と探索範囲を定義 ---
def objective_function(x):
    x_val = x.item()
    return np.sin(x_val) + (x_val / 10)**2

# 探索範囲
bounds = torch.tensor([[-10.0], [10.0]], dtype=torch.float64)

# --- 2. 初期データを生成 ---
initial_x = torch.linspace(bounds[0, 0], bounds[1, 0], 5, dtype=torch.float64).unsqueeze(1)
initial_y = torch.tensor([objective_function(x) for x in initial_x], dtype=torch.float64).unsqueeze(1)
train_X = initial_x
train_Y = initial_y

# --- BoTorch/GPyTorchを使うためのヘルパー関数 ---
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

def get_next_candidate(train_X, train_Y):
    # (a) モデルの学習
    model = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # (b) 獲得関数の定義 (EI)
    acq_func = ExpectedImprovement(model, best_f=train_Y.min())

    # (c) 獲得関数を最大化して、次の候補点を見つける
    candidate, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=1,
        num_restarts=5,
        raw_samples=20,
    )
    return candidate

# --- 3. 最適化ループを実行！ ---
N_ITERATIONS = 15

for i in range(N_ITERATIONS):
    print(f"--- イテレーション {i+1}/{N_ITERATIONS} ---")

    # 次の候補点を取得
    new_candidate = get_next_candidate(train_X, train_Y)

    # (d) 目的関数を評価
    new_y = objective_function(new_candidate)

    # (e) データを更新
    train_X = torch.cat([train_X, new_candidate])
    train_Y = torch.cat([train_Y, torch.tensor([[new_y]], dtype=torch.float64)])

    print(f"試行した点: {new_candidate.item():.4f}, 結果: {new_y:.4f}")
    print(f"現在の最小値: {train_Y.min().item():.4f}")


# --- 4. 最終結果 ---
best_idx = train_Y.argmin()
best_x = train_X[best_idx]
best_y = train_Y[best_idx]
print("\n--- 最適化完了 ---")
print(f"見つかった最小値: {best_y.item():.4f}")
print(f"その時のxの値: {best_x.item():.4f}")

--- イテレーション 1/15 ---


/usr/local/lib/python3.12/dist-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
/usr/local/lib/python3.12/dist-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarni

試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 2/15 ---
試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 3/15 ---


/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
/usr/local/lib/python3.12/dist-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the

試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 4/15 ---
試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 5/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 6/15 ---


...

試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 7/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 8/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 9/15 ---


/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
/usr/local/lib/python3.12/dist-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the

試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 10/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 11/15 ---
試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 12/15 ---


/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)
/usr/local/lib/python3.12/dist-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/usr/local/lib/python3.12/dist-packages/botorch/acquisition/analytic.py:337: NumericsWarning: ExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 ExpectedImprovement 	 --> 	 LogExpectedImprovement 

instead, which fixes the

試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 13/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089
--- イテレーション 14/15 ---
試行した点: -5.0000, 結果: 1.2089
現在の最小値: -0.7089
--- イテレーション 15/15 ---
試行した点: -10.0000, 結果: 1.5440
現在の最小値: -0.7089

--- 最適化完了 ---
見つかった最小値: -0.7089
その時のxの値: 5.0000
